In [1]:
import glob
import os
import fuse, causal_utils
glob.glob("/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/*/")

['/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_3/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_2/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_1/']

In [31]:
!ls /raid/can/nli_models/baseline_mind_distill/nli/seed1/

best.th		      model_state_e2_b0.th  raw_train.jsonl
config.json	      model_state_e3_b0.th  temperature.pt
meta.json	      model.tar.gz	    training_state_e2_b0.th
metrics_epoch_0.json  normal		    training_state_e3_b0.th
metrics_epoch_1.json  out.log		    vocabulary
metrics_epoch_2.json  raw_m.jsonl
metrics.json	      raw_mm.jsonl


In [59]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

# select model
model_path='/raid/can/nli_models/baseline_mind_distill/nli/seed1/'
# select data path
data_path='/ist/users/canu/debias_nlu/data/nli/'
df = pd.read_json(model_path+'raw_train.jsonl', lines=True)
# select fusion method here
fusion = fuse.sum_fuse

#  get score from the bias model

In [60]:
# avg prob
#s score from bias model
df_hans = pd.read_json(data_path+'hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
hans_score=[b for b in df_hans['bias_probs'] ]
hans_score=np.array(hans_score)

# Get avg  prob of bert

In [61]:
list_probs = []
for i in df['probs']:
    list_probs.extend(i)
x=np.array(list_probs)
avg=np.average(x,axis=0)

In [62]:
avg

array([0.29599526, 0.34789062, 0.35611412])

# y1m0

In [63]:
bias_score=fusion(avg,hans_score)
#y1m0

In [64]:
bias_score[0]

array([-0.3057925 , -0.46523137, -0.49389628])

In [65]:
result_path=model_path+'normal/'
# bert model predictions on HANS
df_bert = pd.read_json(result_path+'hans_result.jsonl', lines=True)

In [106]:
# ent = []
y1m1prob = []
for p,h in zip(df_bert['probs'],hans_score):
    new_y1m1 = fusion(np.array(p),h)
    y1m1prob.append(new_y1m1)
  

In [107]:
labels[14000]

'non-entailment'

# corrected y1m0

In [108]:
c = sharpness_correction(hans_score, y1m1prob) 
bias_score = fusion(c,hans_score)

NameError: name 'sharpness_correction' is not defined

# TIE

In [109]:
debias_scores = []
for p,b in zip(y1m1prob,bias_score):
    debias_scores.append(p-b) # TIE  

# Eval HANS

In [110]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
labels = []
for i in debias_scores:
    labels.append(key[np.argmax(i)])
df_bert['debias_label']=labels

In [111]:
text_ans=""
for idx, obj in enumerate(df_bert['label']):
    text_ans = text_ans + "ex"+str(idx)+","+obj+"\n"   
    
text_ans_debias=""
for idx, obj in enumerate(df_bert['debias_label']):
    text_ans_debias = text_ans_debias + "ex"+str(idx)+","+obj+"\n"          

In [112]:
def format_label(label):
    if label == "entailment":
        return "entailment"
    else:
        return "non-entailment"

guess_dict = {}
for line in text_ans.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict[parts[0]] = format_label(parts[1])
        
guess_dict_debias = {}
for line in text_ans_debias.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias[parts[0]] = format_label(parts[1])        

In [113]:
labels,baseline_avg=causal_utils.get_heur(guess_dict) # normal

Heuristic entailed results:
lexical_overlap: 0.9634
subsequence: 0.9958
constituent: 0.9982

Heuristic non-entailed results:
lexical_overlap: 0.6628
subsequence: 0.1176
constituent: 0.2214
avg: 0.6598666666666666


In [114]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias) # TIE

Heuristic entailed results:
lexical_overlap: 0.9644
subsequence: 0.9962
constituent: 0.9986

Heuristic non-entailed results:
lexical_overlap: 0.6568
subsequence: 0.108
constituent: 0.185
avg: 0.6515000000000001


# Causal Mediation Analysis

In [115]:
def get_ans(ans):
    if ans == 0:
        return 'entailment'
    else:
        return 'non-entailment' 
    
# y0m0
import pickle
modelname='mnli_lr_model.sav'
loaded_model = pickle.load(open(modelname, 'rb'))
x0=loaded_model.predict_proba(np.array([[0,0,0.41997876976119086]]))
m0=avg
y0m0=fusion(x0,m0)    

In [123]:
factual_pred_correct = []
TIE_pred_correct = []
NIE_pred_correct = []
INTmed_pred_correct = []
pred_correct = []
all_TIE = []
all_NIE = []
all_NDE = []
all_INTmed = []
for i in range(len(labels)): 
    y1m1 = y1m1prob[i]
    y1m0 = bias_score[i]
    TE = y1m1 - y0m0
    NDE = bias_score[i] - y0m0
    y0m1= fusion(x0,np.array(df_bert['probs'][i]) )
    TIE = y1m1 - y1m0
    NIE = y0m1 - y0m0
    INTmed = TIE - NIE
    # factual
    factual_ans = np.argmax(df_bert['probs'][i])
    factual_ans = get_ans(factual_ans)
    factual_correct = factual_ans==labels[i]   
    factual_pred_correct.append(factual_correct)
    # TIE
    TIE_ans = np.argmax(TIE)
    TIE_ans = get_ans(TIE_ans)
    TIE_correct = TIE_ans==labels[i]  
    TIE_pred_correct.append(TIE_correct)
    # INTmed
    INTmed_ans = np.argmax(INTmed[0])
    INTmed_ans = get_ans(INTmed_ans)
    INTmed_correct = INTmed_ans==labels[i]  
    INTmed_pred_correct.append(INTmed_correct)
    # NIE
    NIE_ans = np.argmax(NIE[0])
    NIE_ans = get_ans(NIE_ans)
    NIE_correct = NIE_ans==labels[i]  
    NIE_pred_correct.append(NIE_correct)    
    
    # save
    all_NDE.append(NDE[0][0])
    all_NIE.append(NIE[0][0])
    all_TIE.append(TIE[0])
    all_INTmed.append((INTmed[0][0]))
    if  (TIE[0]/TE[0][0])<9999999:
        cf_ans = np.argmax(np.array(df_bert['probs'][i]-hans_score[i]))
        cf_ans = get_ans(cf_ans)
        cf_correct = cf_ans==labels[i]
    else:
#         print(cf_ans)
        cf_correct = factual_ans ==labels[i]
    pred_correct.append(cf_correct)
    
#     np.array(df_bert['probs'][i]-bias_score)
#     labels[i]
print(np.array(all_TIE).mean(),(np.array(all_TIE).std()))
print(np.array(all_INTmed).mean(),(np.array(all_INTmed).std()))
print(np.array(all_NIE).mean(),(np.array(all_NIE).std()))

0.05897773440853179 0.04002435525754341
-0.02051590710053735 0.013665625425377402
0.07949364150906912 0.05368544027567681


In [119]:
np.array(df_bert['probs'][i]-hans_score[i])

array([-0.06199987, -0.03410642,  0.09610631])

In [120]:
df_bert['probs'][i]

[0.6700639724731441, 0.141605570912361, 0.18833048641681602]

In [121]:
hans_score[i]

array([0.73206384, 0.17571199, 0.09222417])

In [50]:
print(sum(NIE_pred_correct)/30000,sum(INTmed_pred_correct)/30000)

0.6439333333333334 0.4606


In [51]:
print(sum(factual_pred_correct)/30000,sum(pred_correct)/30000,sum(TIE_pred_correct)/30000)

0.6598666666666667 0.5182 0.6515


In [24]:
#TIE # 0.032416620996870094 # utama best seed  0.04260506151060745
#TIE # 0.0879242620437139 # PoE
#TIE # 0.10864416858494 # Baseline 0.10295758381662952
#
# baseline
# reweight_korn
# poe_korn


In [52]:
!ls /raid/can/nli_models/

baseline	       check_baseline  poe   poe3
baseline_mind_distill  korn_reweight   poe2  reweight_utama_github


In [53]:
!python cma.py

In [57]:
import cma
import glob
import numpy as np
import os
import fuse, causal_utils
model_path='/raid/can/nli_models/baseline_mind_distill/'
task='nli'
data_path='/ist/users/canu/debias_nlu/data/' + task + '/'
fusion = fuse.sum_fuse
test_set='hans'

In [58]:
cma.report_CMA(model_path,task,data_path,test_set,fusion)

/raid/can/nli_models/baseline_mind_distill/nli/seed3/normal/
0.05672240423200177 0.04367789158020039
-0.019779908793118497 0.014807252886696553
0.07650231302512027 0.05848018108650776
/raid/can/nli_models/baseline_mind_distill/nli/seed2/normal/
0.052288068563398654 0.04736277649260647
-0.018279927934217562 0.015949873664305683
0.0705679964976162 0.06330753909230802
/raid/can/nli_models/baseline_mind_distill/nli/seed1/normal/
0.05897773440853179 0.04002435525754341
-0.02051590710053735 0.013665625425377402
0.07949364150906912 0.05368544027567681
factual score:
[0.66, 0.6872, 0.6598666666666667] 0.6690222222222223 0.012853745190881088
TE:
[0.18167084138284345, 0.17734217041887054, 0.18387677539931505] 0.180963262400343 0.0027142545416092663
TIE:
[0.05672240423200177, 0.052288068563398654, 0.05897773440853179] 0.055996069067977404 0.002778918177727792
[0.6533666666666667, 0.6796666666666666, 0.6515] 0.661511111111111 0.012860514744992628
NIE:
[0.07650231302512027, 0.0705679964976162, 0.07

In [56]:
df_bert = pd.read_json('/raid/can/nli_models/poe3/nli/outputs_poe_bert_base_korn_clark_1_seed13370/'+'raw_mm.jsonl', lines=True)

In [5]:
!ls /raid/can/nli_models/

baseline  check_baseline  korn_reweight  poe  poe2  poe3  reweight_utama_github


In [7]:
!ls -ltr /raid/can/nli_models/baseline/nli/outputs_bert_base_clark_1_seed13370/normal/


total 2406868
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 spelling_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 negation_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 numerical_reasoning_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 length_mismatch_result.txt
-rw-rw-r-- 1 canu canu        65 Nov 30 13:43 antonym_result.txt
-rw-rw-r-- 1 canu canu        66 Nov 30 13:43 word_overlap_result.txt
-rw-rw-r-- 1 canu canu 193014347 Dec  1 00:08 distill_output_1.jsonl
drwxrwxr-x 3 canu canu      4096 Dec  1 01:07 distill_model_1
-rw-rw-r-- 1 canu canu 615717404 Dec  1 05:01 raw_train_result_2.jsonl
drwxrwxr-x 4 canu canu      4096 Dec  2 00:10 distill_model_0
-rw-rw-r-- 1 canu canu 193128135 Dec  2 04:52 distill_output_0.jsonl
-rw-rw-r-- 1 canu canu        67 Dec  7 22:36 result.txt
-rw-rw-r-- 1 canu canu        66 Dec  7 22:36 snli_hard_result.txt
-rw-rw-r-- 1 canu canu        66 Dec  7 22:36 mnli_hard_dev_mm_result.txt
-rw-rw-r-- 1 canu canu        52 Dec  

In [22]:
import pandas as pd

In [23]:
  df_bias_model = pd.read_json(data_path+'test'+'_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)

In [34]:
if '-' in df_bias_model['gold_label'].value_counts():
    print(df_bias_model['gold_label'].value_counts()['-'])

168


In [9]:
df_bias_model

NameError: name 'df_bias_model' is not defined

In [124]:
import torch

In [125]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [126]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7fa37bdb2490>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fa37bdb26a0>


In [127]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836],
        [0.1463, 0.1088, 0.0836]])
tensor([0.1463, 0.1088, 0.0836])


NameError: name 'model' is not defined

In [131]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [166]:
import os
import pandas as pd
from torchvision.io import read_image
import torch.nn as nn

class CustomImageDataset(Dataset):
    def __init__(self, probs,target_probs):
        self.probs  = torch.tensor(probs)
        self.target_probs = torch.tensor(target_probs)
    def __len__(self):
        return len(self.probs)

    def __getitem__(self, idx):
        return self.probs[idx], self.target_probs[idx]


In [169]:
df_bert['probs'][0]

[0.13495792448520602, 0.7968678474426271, 0.068174198269844]

In [170]:
train=CustomImageDataset(df_bert['probs'],df_bert['probs'])

In [171]:
class TempScale(nn.Module):
    def __init__(self):
        super(TempScale, self).__init__()
        self.T = nn.Parameter(torch.tensor(1.0))

    def forward(self, x):
        x = (x/self.T)/torch.sum(x/self.T)
        return x
model = TempScale()

In [172]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = dataloader.__len__()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [173]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_loop(train, model, loss_fn, optimizer)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [174]:
train.__len__()

30000

In [165]:
torch.tensor(df_bert['probs']).shape

torch.Size([30000, 3])